<h1 style="text-align: center;"> Tarea Machine Learning </h1>
<h1 style="text-align: right; font-size: 24px; margin-right: 10px;">Guillermo Díaz Aguado</h1>

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [20]:
df = pd.read_excel("datos_tarea25.xlsx")

In [21]:
def analizar_variables_categoricas(datos):
    """
    Analiza variables categóricas en un DataFrame.

    Args:
        datos (DataFrame): El DataFrame que contiene los datos.

    Returns:
        dict: Un diccionario donde aparecen las diferentes categorias, sus frecuencias
        absolutas y relativas.
    """
    # Inicializar un diccionario para almacenar los resultados
    resultados = {}
    
    # Genera una lista con los nombres de las variables.
    variables = list(datos.columns) 
    
    # Seleccionar las columnas numéricas en el DataFrame
    numericas = datos.select_dtypes(include=['int', 'int32', 'int64','float', 'float32', 'float64']).columns

    # Seleccionar las columnas categóricas en el DataFrame
    categoricas = [variable for variable in variables if variable not in numericas]
    
    # Iterar a través de las variables categóricas
    for categoria in categoricas:
        # Verificar si la variable categórica existe en el DataFrame
        if categoria in datos.columns:
            # Crear un DataFrame de resumen para la variable categórica
            resumen = pd.DataFrame({
                'n': datos[categoria].value_counts(),             # Conteo de frecuencias
                '%': datos[categoria].value_counts(normalize=True)  # Porcentaje de frecuencias
            })
            resultados[categoria] = resumen  # Almacenar el resumen en el diccionario
        else:
            # Si la variable no existe en los datos, almacenar None en el diccionario
            resultados[categoria] = None
    
    return resultados

analizar_variables_categoricas(df)

{'Levy':          n         %
 Levy                
 -      644  0.148387
 1017   155  0.035714
 891    145  0.033410
 394    127  0.029263
 779    105  0.024194
 ...    ...       ...
 1848     1  0.000230
 1604     1  0.000230
 2705     1  0.000230
 2959     1  0.000230
 11706    1  0.000230
 
 [203 rows x 2 columns],
 'Manufacturer':                   n         %
 Manufacturer                 
 TOYOTA         1547  0.356452
 HYUNDAI        1234  0.284332
 MERCEDES-BENZ   932  0.214747
 LEXUS           627  0.144470,
 'Category':               n         %
 Category                 
 Sedan      2317  0.533871
 Jeep       1510  0.347926
 Hatchback   513  0.118203,
 'Leather interior':                      n         %
 Leather interior                
 Yes               3789  0.873041
 No                 551  0.126959,
 'Fuel type':               n         %
 Fuel type                
 Petrol     2172  0.500461
 Hybrid     1248  0.287558
 Diesel      920  0.211982,
 'Engine volume':     

In [ ]:


# Quito los "-" de la columna "Levy"
df["Levy"] = df["Levy"].replace("-", np.nan)


def poner_1_y_0(df, columna, str1, str0):
    return df[columna].map({str1:1, str0:0})

# Cambiar las columnas "Leather interior", "Gear box type", "Wheel", "Color" ->  a valores de 0 y 1
df["Leather interior"] = poner_1_y_0(df, "Leather interior", "Yes", "No")
df["Gear box type"] = poner_1_y_0(df, "Gear box type", "Tiptronic", "Automatic")
df["Wheel"] = poner_1_y_0(df, "Wheel", "Left wheel", "Right-hand drive")
df["Color"] = poner_1_y_0(df, "Color", "White", "Black")


# Crear columna de 1 y 0 que indica si tiene turbo
df["Tiene Turbo"] = df["Engine volume"].astype(str).str.contains("Turbo").astype(int)

# Limpiar la columna "Engine volume" y convertir a numérico
df["Engine volume"] = df["Engine volume"].astype(str).str.replace(r"Turbo", "", regex=True).astype(float)

# Limpiar la columna "Mileage" quitando "km"
df["Mileage"] = df["Mileage"].astype(str).str.replace(r"km", "", regex=True).astype(int)



In [17]:
df.dtypes

Price                 int64
Levy                 object
Manufacturer         object
Prod. year            int64
Category             object
Leather interior      int64
Fuel type            object
Engine volume       float64
Mileage               int32
Cylinders             int64
Gear box type         int64
Drive wheels         object
Wheel                 int64
Color                 int64
Airbags               int64
Tiene Turbo           int32
dtype: object

In [18]:
df.nunique()

Price                915
Levy                 202
Manufacturer           4
Prod. year            29
Category               3
Leather interior       2
Fuel type              3
Engine volume         40
Mileage             1826
Cylinders             10
Gear box type          2
Drive wheels           3
Wheel                  2
Color                  2
Airbags               16
Tiene Turbo            2
dtype: int64

In [19]:
df["Wheel"]

0       1
1       1
2       1
3       1
4       1
       ..
4335    1
4336    1
4337    1
4338    1
4339    1
Name: Wheel, Length: 4340, dtype: int64